In [1]:
import keras
from keras.layers import Input, Dense, LSTM
from keras.models import Model, Sequential

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
filename = 'glove.50000.txt'
def loadGloVe(filename):
    vocab = set()
    embd = {}
    word = None
    with open(filename) as f:
        try:
            for line in f:
                row = line.strip().split(' ')
                word = row[0]
                vocab.add(row[0])
                embd[word] = np.array([float(x) for x in row[1:]])
            
        except Exception as e:
            print(word)
    print('Loaded GloVe!')
    return vocab,embd
vocab_set,embd = loadGloVe(filename)
# vocab_size = len(vocab)
# vocab_set = set(vocab)
# embedding_dim = len(embd[0])
# embedding = np.asarray(embd)

Loaded GloVe!


In [4]:
vocab_set.add('<UNK>')

In [5]:
embd['<UNK>'] = np.zeros(50, dtype = np.float32)

In [6]:
vocab_list = list(vocab_set)
word_to_index = {vocab_list[i]: i for i in range(len(vocab_list))}

In [7]:
('<UNK>' in vocab_set)

True

In [9]:
# import json
# import os
# import numpy
# def generate_vocabulary():
#     i = 0
#     vocab = set() 
#     with open("tip.json",encoding="utf-8") as f:
#         for line in f:
#             tip = json.loads(line.strip())['text']
# #             print (tip)
#             words = tip.lower().split()
#             vocab |= set(words)
#             i +=1
#             if i == 5000:
#                 break
#     vocab.add('start')
    
#     return vocab,list(vocab)
# # def generate_data():
# vocab_set,vocab_list = generate_vocabulary()

In [8]:
# word_index = {vocab_list[i]:i for i in range(len(vocab_list))}
transnet_size = 64
sequence_length = 3
def one_hot(word,word_index,vocab_len):
    x = [0]*vocab_len
    if word in word_index:
        x[word_index[word]] = 1
    return x
# def one_hot_(k,l):
#     x = [0]*(l)
#     while k > 0:        
#         x[k%l] = 1
#         k = k//l
#     return x


In [11]:

# def generate_train():
#     j = 0
#     X = []
#     Y = []
#     total = 1000
#     tnet_file = open('../src/ex2_data/train_epochs/newfile.txt')
#     with open("./data/train_lex_data.txt") as f:
#         for line in f:
#             user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
#             #vec = one_hot_(j,transnet_size)
#             words = ['start' for _ in range(sequence_length)] + tip.strip().lower().split()
#             vectors = []
#             onehot = []
#             for word in words:
#                 encoding = one_hot(word)
#                 onehot += [encoding]
#                 vectors += [encoding+vec]
#             for i in range(sequence_length-1,len(words)-1):
#                 X += [vectors[i-sequence_length+1:i+1]]
#                 Y += [onehot[i+1]]
#             j += 1
#             if j % 100 == 0:
#                 print(j)
#             if j == total:
#                 break
#     return np.array(X),np.array(Y)
                    
                
            
            

In [12]:
# X,Y =generate_train()

In [9]:
import pickle
import string

In [10]:
punc = set(string.punctuation)


In [11]:
user_item_index = pickle.load(open('user_item_index_dict.pkl','rb'))

In [16]:
# user_item_index

In [12]:
import pickle
def new_generate_train():
    sequence_length = 3
    j = 0
    X = []
    Y = []
    total = 500
    tnet_file = pickle.load(open('./gen_review_vec_train.pkl','rb'))
    with open("./data/train.txt") as f:
        for line in f:
#             rev_emb = np.zeros(50)
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            rev_index = user_item_index[(user_id,item_id)]
            
            #vec = one_hot_(j,transnet_size)
            rev_words = [w.lower() for w in tip.split()]
            words = []
            for w in rev_words:
                prev = 0
                for ch in range(len(w)):
                    if w[ch] in punc:
                        if prev < ch:
                            words += [w[prev:ch]]
                        words += [w[ch]]
                        prev = ch + 1
                if prev < len(w):
                    words += [w[prev:]]
            if len(words)< 5:
                words += ['<UNK>']*(5-len(words))
            elif len(words) > 5:
                words = words[:5]
            words = ['<UNK>' for _ in range(sequence_length)] + words
            vectors = []
            onehot = []
            vec = tnet_file[rev_index] #index of review from tnet file
            j += 1
#             if j % 10 == 0:
#                 print(j,end=" ")
            for word in words:
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                onehot += [onehot_encoding]
                vectors += [np.concatenate([encoding,vec])]
            X = []
            Y = []
            for i in range(sequence_length-1,len(words)-1):
                X += [vectors[i-sequence_length+1:i+1]]
                Y += [onehot[i+1]]
#             X,Y = np.array(X),np.array(Y)
#             print(X.shape,Y.shape)
            yield X,Y
#                 Y += []
            
#             if j == total:
#                 break
#     return X,Y#np.array(X),np.array(Y)
                    
                
            
            

In [13]:
X =[]
Y = []
for x,y in new_generate_train():
    X += x
    Y += y
X = np.array(X)
Y = np.array(Y)

In [14]:
X.shape

(2500, 3, 114)

In [15]:
Y.shape

(2500, 50001)

In [139]:
# i=0
# for x,y in new_generate_train():
#     if i % 10 == 0:
#         print(i,end=" ")
#     i += 1
#     if i == 600:
#         break

In [16]:
sequence_length = 3
transnet_size = 64
word_emb_size = 50

In [141]:
# keras.backend.clear_session()

In [28]:
V = len(vocab_list)
model = Sequential()
model.add(LSTM(15,input_shape=(sequence_length,word_emb_size+transnet_size)))
model.add(Dense(V,activation="softmax"))

In [29]:
len(vocab_list)

50001

In [30]:
# model.summary()
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [31]:
# X_train, Y_train = a,b #generate_train()

In [32]:
import tensorflow as tf
# keras.backend.get_session().run(tf.global_variables_initializer())
model.fit(X,Y,nb_epoch=500)

Epoch 1/500
2500/2500 [==============================] - 11s 4ms/step - loss: 7.8894 - acc: 0.0284
Epoch 2/500
2500/2500 [==============================] - 11s 4ms/step - loss: 6.1491 - acc: 0.0360
Epoch 3/500
2500/2500 [==============================] - 10s 4ms/step - loss: 5.8067 - acc: 0.0544
Epoch 4/500
2500/2500 [==============================] - 10s 4ms/step - loss: 5.5893 - acc: 0.0648
Epoch 5/500
2500/2500 [==============================] - 10s 4ms/step - loss: 5.3899 - acc: 0.0788
Epoch 6/500
2500/2500 [==============================] - 10s 4ms/step - loss: 5.2143 - acc: 0.0884
Epoch 7/500
2500/2500 [==============================] - 10s 4ms/step - loss: 5.0589 - acc: 0.1056
Epoch 8/500
2500/2500 [==============================] - 10s 4ms/step - loss: 4.9320 - acc: 0.1080
Epoch 9/500
2500/2500 [==============================] - 10s 4ms/step - loss: 4.8352 - acc: 0.1180
Epoch 10/500
2500/2500 [==============================] - 10s 4ms/step - loss: 4.7740 - acc: 0.1212
Epoch 11/

In [33]:

#sample
import numpy as np
def long_to_small(preds):
    preds = preds.tolist()
    index = []
    small_preds = []
    for i in range(len(preds)):
        if preds[i] > 0.001:
            small_preds += [preds[i]]
            index += [i]
    return small_preds,index
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    #print preds[:10], np.max(preds)
    #print(len(preds))
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [34]:
# X_test,Y_test = generate_data(False)

In [35]:
# re  = model.predict(X_test)

In [36]:
# for i in range(len(re)):
#     print(vocab_list[sample(re[i])],end=" ")

In [37]:
out=[] 

In [38]:
import pickle
def test(out):
    sequence_length = 3
    
    j = 0
    X = []
    Y = []
    #total = 1000
    tnet_file = pickle.load(open('./gen_review_vec_test.pkl','rb')) #changr to test
    with open("./data/Old/test_lex_data.txt") as f:
        for line in f:
            rev_emb = np.zeros(50)
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            #vec = one_hot_(j,transnet_size)
#             rev_words = [w.lower() for w in review_text.split()]
            words = []
            words = ['<UNK>' for _ in range(sequence_length)] + words
            vectors = []
            onehot = []
            vec = tnet_file[j]
            for word in words:
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                onehot += [onehot_encoding]
                vectors += [np.concatenate([encoding,vec])]
            #for i in range(sequence_length-1,len(words)-1):
            X += [vectors[0:sequence_length]]
#                 Y += [onehot[i+1]]
            j += 1
            yp = model.predict(np.array(X))[0]
            yp_small, index = long_to_small(yp)
            yp = vocab_list[index[sample(yp_small)]]
            generated_tip = [yp]
            words += [yp]
            for i in range(sequence_length,10+sequence_length):
                word = generated_tip[-1]
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                onehot += [onehot_encoding]
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                vectors += [np.concatenate([encoding,vec])]
                X += [vectors[i-sequence_length+1:i+1]]
                x = [X[-1]]
                yp = model.predict(np.array(x))[0]
                yp_small, index = long_to_small(yp)
                yp = vocab_list[index[sample(yp_small)]]
                generated_tip += [yp]
                words += [yp]
            out += [" ".join(generated_tip)]
            if j == 5:
                break

In [39]:
test(out)

In [40]:
out

['great this is we <UNK> i . . live locals good',
 'super , . . one . . . with essen head',
 'location and . get fresh . and they nice ! go',
 'nice sushi really need spot ! clean great afraid some i',
 'location shannon and awesome delicious if , spot can tea my']

In [27]:
# for 150 epochs
out

['julie food . good food . die vogel favorite with lots',
 "an town this priced the terrible product down ' this not",
 'this " - selection a amazing time . . my a',
 'hoppy great place . fresh and 32 ink see down are',
 'a week to - photo hooked slim right a fresh and']

##### Trying

In [47]:
import pickle
def test(out):
    sequence_length = 3
    
    j = 0
    X = []
    Y = []
    #total = 1000
    tnet_file = pickle.load(open('./gen_review_vec_test.pkl','rb')) #changr to test
    with open("./data/Old/test_lex_2000.txt") as f:
        for line in f:
            rev_emb = np.zeros(50)
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            #vec = one_hot_(j,transnet_size)
#             rev_words = [w.lower() for w in review_text.split()]
            words = []
            words = ['<UNK>' for _ in range(sequence_length)] + words
            vectors = []
            onehot = []
            vec = tnet_file[j]
            for word in words:
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                onehot += [onehot_encoding]
                vectors += [np.concatenate([encoding,vec])]
            #for i in range(sequence_length-1,len(words)-1):
            X += [vectors[0:sequence_length]]
#                 Y += [onehot[i+1]]
            j += 1
            yp = model.predict(np.array(X))[0]
            yp_small, index = long_to_small(yp)
            yp = vocab_list[index[sample(yp_small)]]
            generated_tip = [yp]
            words += [yp]
            for i in range(sequence_length,10+sequence_length):
                word = generated_tip[-1]
                onehot_encoding = one_hot(word,word_to_index,len(vocab_set))
                onehot += [onehot_encoding]
                if word in embd:
                    encoding = embd[word]
                else:
                    encoding = embd['<UNK>']
                vectors += [np.concatenate([encoding,vec])]
                X += [vectors[i-sequence_length+1:i+1]]
                x = [X[-1]]
                yp = model.predict(np.array(x))[0]
                yp_small, index = long_to_small(yp)
                yp = vocab_list[index[sample(yp_small)]]
                generated_tip += [yp]
                words += [yp]
            out += [" ".join(generated_tip)]
            if j%50 == 0:
                print j,

In [48]:
out2 = []

In [49]:
test(out2)

50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000


In [60]:
i = 0
fopen = open('./Result_RNN.txt','w')
with open('./data/Old/test_lex_2000.txt') as f:
    for line in f:
        _,_,_,_,_,tip = line.split("\t")
        fopen.write(out2[i] + "\t" + tip)
        i += 1
fopen.close()

In [63]:
out2[5]

'beef great favorite open . lots to poutine i it .'

In [62]:
rnn_out = open('RNN_out.txt','w')
for line in out2:
    rnn_out.write(line + "\n")

##### Old code

In [30]:
def generate_test_data():
    with open('tip.json') as f:
        i = 0
        for line in f:
            tip = json.loads(line.strip())['text']
            vec = one_hot_(i,transnet_size)
            X = []
            one_hot_v = one_hot('start')
            X = [one_hot_v+vec for _ in range(sequence_length)]
                
            return vec,tip,numpy.array(X).reshape(1,sequence_length,-1)

In [31]:
vec,tip,X = generate_test_data()

In [32]:
for i in range(10):
    y = model.predict(X)
#     print(y.shape)
#     print (y)
    w = sample(y[0])
#     print(w)
    print(vocab_list[w],end=" ")
    X = numpy.delete(X,0,1)
#     print(X.shape)
    X = numpy.insert(X,sequence_length-1,one_hot_(w,len(vocab_list))+vec,axis=1)
#     print(X.shape)
#     print(X[0][sequence_length-1])
# X.shape

the kidding. has even this up be people to easy 

In [33]:
tip

'Get here early enough to have dinner.'

In [ ]:
# import tensorflow as tf
# tf.__version__

In [78]:
pickle.load(open('./gen_review_vec_train.pkl','rb'),encoding="bytes")

array([[ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       [ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       [ 0.0677953 ,  0.12908284,  0.00995188, ...,  0.09868402,
         0.05035567,  0.0261237 ],
       ..., 
       [ 0.06633914,  0.1292228 ,  0.00890549, ...,  0.09720803,
         0.04496233,  0.02088072],
       [ 0.07077023,  0.13585889,  0.01115206, ...,  0.09501065,
         0.03852132,  0.01822961],
       [ 0.06911414,  0.13658382,  0.01111554, ...,  0.09772882,
         0.04012528,  0.02351698]], dtype=float32)

In [129]:
file = open ('data/train.txt','w',encoding='utf-8')
i = 0
with open("./data/train_lex_data.txt",encoding='utf-8') as f:
        for line    in f:
            user_id,item_id,rating,review_id,review_text,tip = line.strip().split('\t')
            if len(tip) < 25:
                continue
            file.write(line)
            i += 1
            if i == 500:
                break
file.close()